In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import copy
import time
import math
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import concurrent.futures
import psycopg2
from psycopg2.errors import UniqueViolation
import config
import ulta_functions as ulta

In [114]:
def get_products_in_stock(secret_sales, driver):
    wait = WebDriverWait(driver, 30)
    products_in_stock = {}
    for product_id in secret_sales:
        temp = {}
        driver.get(secret_sales[product_id]['url'])
        time.sleep(1)
        product_variants = driver.find_elements_by_class_name('ProductSwatchImage__variantHolder')
        if len(product_variants) == 0:
            product_variants = driver.find_elements_by_class_name('ProductDetail__productSwatches')
        for product_variant in product_variants:
            try:
                product_variant.click()
            except:         
                next
            else:
                wait.until(EC.presence_of_element_located((By.XPATH, "/html/head/meta[10]")))
                soup = BeautifulSoup(driver.page_source, features="lxml")
                price = soup.find('meta', {'property' : 'product:price:amount'}).get('content')
                option = ulta.get_option(soup)
                temp[option] = price
        variants_in_stock = ulta.rearrange_product_dict(temp)
        products_in_stock[product_id] = variants_in_stock
    return(products_in_stock)

In [115]:
def add_row_extra_tbl(conn, product_id, rating, no_of_reviews, offers, date, price, options):
    cur = conn.cursor()
    cur.execute("rollback;")
    cur.execute("SELECT products_pkey FROM products WHERE product_id=%s", (product_id,))
    products_pkey = cur.fetchone()[0]
    
    sql = """INSERT INTO extra (rating, no_of_reviews, offers, date, price, options, products_pkey_foreign) VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    try:
        cur.execute(sql, (rating, no_of_reviews, offers, date, price, options, products_pkey))
    except UniqueViolation:
        cur.close()
        return
    else:
        cur.close()

In [116]:
def add_row_products_tbl(conn, product_id, brand, product, main_category, sub_category, sub_sub_category, url):
    cur = conn.cursor()
    cur.execute("rollback;")
    sql = """INSERT INTO products (product_id, brand, product, main_category, sub_category, sub_sub_category, url) VALUES (%s, %s, %s, %s, %s, %s, %s) RETURNING products_pkey"""
    try:
        cur.execute(sql, (product_id, brand, product, main_category, sub_category, sub_sub_category, url))
        products_pkey = cur.fetchone()[0]
    except UniqueViolation:
        cur.execute("rollback;")
        query = """SELECT products_pkey FROM products WHERE (product_id = %s)"""
        cur.execute(query, (product_id,))
        products_pkey = cur.fetchone()[0]
    finally:
        cur.close()
        return(products_pkey)

In [117]:
def add_row_ratings_and_offers_tbl(conn, rating, no_of_reviews, offers, date, products_pkey):
    cur = conn.cursor()
    cur.execute("rollback;")
    sql = """INSERT INTO ratings_and_offers (rating, no_of_reviews, offers, date, products_pkey_foreign) VALUES (%s, %s, %s, %s, %s) RETURNING ratings_and_offers_pkey"""
    try:
        cur.execute(sql, (rating, no_of_reviews, offers, date, products_pkey))
        ratings_and_offers_pkey = cur.fetchone()[0]
    except UniqueViolation:
        cur.execute("rollback;")
        query = """SELECT ratings_and_offers_pkey FROM ratings_and_offers WHERE (products_pkey_foreign = %s AND date = %s)"""
        cur.execute(query, (products_pkey, date))
        ratings_and_offers_pkey = cur.fetchone()[0]
    finally:
        cur.close()
        return(ratings_and_offers_pkey)    

In [118]:
def add_rows_prices_tbl(conn, ratings_and_offers_pkey, ratings_and_offers_id, df):
    cur = conn.cursor()
    cur.execute("rollback;")
    query = "ratings_and_offers_id == '{idd}'".format(idd = ratings_and_offers_id)
    filtered_df = df.query(query)
    df_rows_tuple = ()
    for i in range(len(filtered_df)):
        row = (ratings_and_offers_pkey, filtered_df.iloc[i]['price'], filtered_df.iloc[i]['price_str'], str(filtered_df.iloc[i]['option']))
        df_rows_tuple = (row,) + df_rows_tuple
    sql = """INSERT INTO prices (ratings_and_offers_pkey_foreign, price, price_str, option) VALUES (%s, %s, %s, %s)"""
    try:
        cur.executemany(sql, df_rows_tuple)
    except UniqueViolation:
        cur.execute("rollback;")
        cur.close()
    else:
        cur.close()

### ulta_df

In [7]:
ulta_df = pd.read_csv('data/ulta_df.csv').rename_axis('ulta_id').drop(columns={'sale'})#.fillna(value={'offers': ' ', 'options': ' '})

In [8]:
len(ulta_df)

16881

## 2nd attempt: 2 tables

### products

In [9]:
products_df = (
    ulta_df
    .pipe(copy.deepcopy)
    .loc[:, ['product_id', 'main_category', 'sub_category', 'sub_sub_category', 'brand', 'product', 'url']]
)

products_df.to_csv('data/products_df.csv')

### ratings_and_offers

In [10]:
fixed_sale_prices = (
    ulta_df
    .pipe(copy.deepcopy)
    .dropna(subset=['sale_price'])
    .loc[:, ['price', 'sale_price']]
    .rename(columns={'price' : '01/01/1911', 'sale_price' : '08/13/2020'})
    .reset_index()
    .pipe(pd.melt, id_vars=['ulta_id'], var_name='date', value_name='price')
    .pipe(pd.merge, ulta_df.loc[:, ['options', 'rating', 'no_of_reviews', 'offers', 'url', 'product_id']], on='ulta_id', how='left')
    .set_index('ulta_id')
)

no_sale_price = ulta_df[pd.isnull(ulta_df['sale_price'])].loc[:, ['price', 'options', 'rating', 'no_of_reviews', 'offers', 'url', 'product_id']]
no_sale_price['date'] = ['08/13/2020'] * len(no_sale_price)

#I'm bad at naming tables
extra_df = (
    pd.concat([fixed_sale_prices, no_sale_price])
    .reset_index()
    .rename_axis('extra_id')
)

extra_df.to_csv('data/extra_df.csv')

## 1st attempt: 3 tables

I originally wanted to make a third table but I failed but I don't want to delete my code

In [121]:
ulta_df = pd.read_csv('data/ulta_df_823.csv').rename_axis('ulta_id')

In [157]:
urls = []
for i in range(len(ulta_df)):
    if ulta_df.iloc[i]['url'][:41] == 'https://www.ulta.comhttps://www.ulta.com/':
        url = ulta_df.iloc[i]['url'][20:]
    else:
        url = ulta_df.iloc[i]['url']
    urls.append(url)
ulta_df['url'] = urls

### products

In [158]:
products = ulta_df.loc[:, ['product_id', 'main_category', 'sub_category', 'sub_sub_category', 'brand', 'product', 'url']]

In [159]:
products.to_csv('data/products.csv')

### ratings_and_offers

In [160]:
ratings_and_offers = (
    ulta_df
    .loc[:, ['rating', 'no_of_reviews', 'options', 'offers']]
    .reset_index()
    .rename_axis('ratings_and_offers_id')
)
ratings_and_offers['date'] = ['08/23/2020'] * len(ulta_df)

In [161]:
ratings_and_offers

,ulta_id,rating,no_of_reviews,options,offers,date
ratings_and_offers_id,,,,,,
0,0,4.4,832,3 Colors,,08/23/2020
1,1,4.1,10,4 Colors,,08/23/2020
2,2,4.1,523,2 Colors,Online Only Purchase $50 in Clinique product a...,08/23/2020
3,3,4.0,4,4 Colors,,08/23/2020
4,4,3.6,390,6 Colors,,08/23/2020
...,...,...,...,...,...,...
1554,1554,0.0,,,,08/23/2020
1555,1555,0.0,,,,08/23/2020
1556,1556,0.0,,,,08/23/2020


In [149]:
ratings_and_offers.to_csv('data/ratings_and_offers.csv')

### prices

In [165]:
prices = (
    ulta_df
    .loc[:, ['product_id', 'url', 'price', 'options', 'sale', 'sale_price']]
    .rename_axis('ratings_and_offers_id')
)

In [166]:
prices_str = []
mult_sizes = []
for i in range(len(prices)):
    if prices.iloc[i]['sale'] == 0:
        price_raw = prices.iloc[i]['price']
    else:
        price_raw = prices.iloc[i]['sale_price']
    prices_str.append(price_raw)
    if pd.isna(prices.iloc[i]['options']):
        mult_sizes.append(False)
    elif 'Sizes' in prices.iloc[i]['options'] and prices.iloc[i]['options'] != '2 Sizes':
        mult_sizes.append(True)
    else:
        mult_sizes.append(False)

In [167]:
prices['price_str'] = prices_str
prices['mult_sizes'] = mult_sizes
prices = prices.drop(columns={'sale', 'sale_price'})

In [168]:
price = []
one_price_df = (
    prices.query("mult_sizes == False & options != '2 Sizes'")
    .pipe(copy.deepcopy)
    .drop(columns={'url', 'product_id', 'mult_sizes'})
    .rename(columns={'options' : 'option'})
    .reset_index()
)
for i in range(len(one_price_df)): 
    price.append(float(one_price_df.iloc[i]['price_str'].split(' - ')[0][1:]))
one_price_df['price'] = price

In [169]:
two_sizes_df = (
    prices
    .query("options == '2 Sizes'")
    .pipe(copy.deepcopy)
)
two_sizes_df = (
    two_sizes_df.price.str.split(' - ', expand=True)
    .rename(columns={0 : 'size 1', 1 : 'size 2'})
    .reset_index()
    .pipe(pd.melt, id_vars=['ratings_and_offers_id'], var_name='option', value_name='price_split')
    .set_index('ratings_and_offers_id')
    .pipe(pd.merge, prices.loc[:, ['price']], on='ratings_and_offers_id')
    .rename(columns={'price' : 'price_str'})
)
two_sizes_df['price'] = pd.to_numeric(two_sizes_df['price_split'].str[1:])
two_sizes_df = (
    two_sizes_df
    .drop(columns={'price_split'})
    .reset_index()
)

In [170]:
mult_sizes_df = (
    prices
    .query('mult_sizes == True')
    .pipe(copy.deepcopy)
    .reset_index()
)

mult_sizes = (
    mult_sizes_df
    .set_index('product_id')
    .transpose()
    .pipe(pd.DataFrame.to_dict)
)

In [173]:
driver = webdriver.Chrome(r'C:\Users\elerm\Downloads\chromedriver_win32\chromedriver.exe')
products_in_stock = get_products_in_stock(mult_sizes, driver)
driver.close()
driver.quit()

In [174]:
mult_sizes_df = (
    pd.DataFrame.from_dict(products_in_stock)
    .transpose()
    .rename_axis('product_id')
    .reset_index()
    .pipe(pd.melt, id_vars=['product_id'], var_name='price', value_name='option')
    .dropna()
    .pipe(pd.merge, mult_sizes_df, on='product_id')
    .drop(columns={'url', 'price_y', 'mult_sizes', 'options'})
    .rename(columns={'price_x' : 'price'})
    .astype({'price': 'float64'})
)

In [175]:
prices_df = pd.concat([one_price_df, two_sizes_df, mult_sizes_df])

In [142]:
prices.to_csv('data/prices.csv')

## add tables to database

### connect to database

In [177]:
params = config.config()
conn = psycopg2.connect(**params)

In [178]:
for i in range(len(products)):
    products_pkey = add_row_products_tbl(
        conn, 
        products.iloc[i]['product_id'], 
        products.iloc[i]['brand'], 
        products.iloc[i]['product'], 
        products.iloc[i]['main_category'], 
        products.iloc[i]['sub_category'], 
        products.iloc[i]['sub_sub_category'], 
        products.iloc[i]['url']
    )
    ulta_id = products.iloc[i].name
    rao = ratings_and_offers.query("ulta_id == '{idd}'".format(idd = ulta_id))
    ratings_and_offers_id = rao.iloc[0].name
    ratings_and_offers_pkey = add_row_ratings_and_offers_tbl(
        conn,
        rao.iloc[0]['rating'],
        rao.iloc[0]['no_of_reviews'],
        rao.iloc[0]['offers'],
        rao.iloc[0]['date'],
        products_pkey
    )
    add_rows_prices_tbl(
        conn, 
        ratings_and_offers_pkey, 
        ratings_and_offers_id, 
        prices_df
    )

### write them to database

In [ ]:
#for i in range(len(products)):
#    products_pkey = add_row_products_tbl(conn, products.iloc[i]['product_id'], products.iloc[i]['brand'], products.iloc[i]['product'], products.iloc[i]['main_category'], products.iloc[i]['sub_category'], products.iloc[i]['sub_sub_category'], products.iloc[i]['url'])
#    add_rows_extra_tbl(conn, products_pkey, products.iloc[i].name, extra)
#print('DONE!')

### commit the changes and close connection

In [179]:
conn.commit()
conn.close()